## Time calculation

Let's check for segment signal, how many time we will use for feature extraction and predict its labels:

In [1]:
import os
import time
import joblib
import random
import numpy as np
import pandas as pd

# our own pipeline
from pipelines.tools import generate_feature_dict
from pipelines.ml_functions import set_seed
from pipelines.data_prapare import read_seg_signal_data, read_seg_features_table

In [2]:
import warnings
warnings.filterwarnings("ignore")

set_seed(42)

In [3]:
def str_2_list(strr):
    strr = strr.replace("\n","").replace("  ", ",").replace(" ", ",").replace(",,",',').replace("[","").replace("]","")
    results = [float(n) for n in strr.split(',')]
    return results

In [4]:
aat_vis, aat_img, asl_vis, asl_img = read_seg_signal_data()
aat_vis_fe, aat_img_fe, asl_vis_fe, asl_img_fe = read_seg_features_table()
col_name = list(aat_vis_fe.columns)[1:]
asl_vis.head()

,label_index,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16
0,9,[-3907.68842789 -3905.09562553 -3906.74965462 ...,[-5623.90007066 -5623.20716658 -5625.28587881 ...,[-2272.39010005 -2270.80312619 -2274.06648088 ...,[-1734.71888718 -1735.30003253 -1733.93657612 ...,[967.13763084 971.47386926 971.69738671 967.98...,[-2937.89093946 -2932.10183765 -2931.00660217 ...,[3050.11904837 3054.49999028 3056.75751647 304...,[-704.54933698 -700.6377817 -698.98375261 -70...,[233.55337781 233.37456386 233.3969156 230.53...,[8908.31010441 8897.29069439 8913.27219168 890...,[6105.20018401 6107.32359973 6104.06024504 610...,[1041.85951255 1050.88961731 1027.01795423 105...,[-3401.57787819 -3413.26784054 -3400.25912526 ...,[2678.02955842 2663.32211057 2677.56017179 266...,[197.63412447 194.37076978 197.90234541 195.95...,[1246.66854699 1241.01355565 1248.36727957 124...
1,9,[-3919.311335 -3912.33759073 -3906.74965462 ...,[-5656.06423093 -5652.37619309 -5642.11674239 ...,[-2294.83125148 -2293.40073984 -2294.78654799 ...,[-1727.18634929 -1728.90743362 -1723.27479402 ...,[959.06865109 962.30965403 964.50012499 958.64...,[-2921.37300031 -2918.46727353 -2911.60528798 ...,[3044.86638842 3046.54276926 3050.29786233 304...,[-697.93322062 -697.12855782 -692.47939497 -70...,[183.19489756 190.4592145 190.72743544 184.82...,[8853.74949619 8843.02065885 8856.90109216 884...,[6058.14976193 6067.82806728 6064.45295387 606...,[1032.15885546 1049.21323648 1024.51455885 105...,[-3445.94609093 -3445.78962872 -3435.21725359 ...,[2636.45531374 2626.30762175 2640.23275855 262...,[188.91694414 188.15698482 190.6603802 188.67...,[1260.16900065 1250.33423309 1259.18552389 125...
2,9,[-3945.68639346 -3939.20438757 -3950.7825912 ...,[-5681.43346088 -5678.90771376 -5682.59575159 ...,[-2334.32678393 -2330.52698738 -2336.83017931 ...,[-1705.4157502 -1708.23207 -1702.95705831 ...,[948.51862771 949.27858702 952.49723822 945.94...,[-2888.89591562 -2889.25354353 -2878.65881665 ...,[3041.55833024 3039.68078371 3046.74393496 303...,[-692.21117404 -694.28988627 -685.63976117 -69...,[147.23094073 154.85288559 148.39323144 145.86...,[8701.69057866 8695.72266289 8705.15509905 869...,[6026.27617434 6036.75914249 6027.61727901 602...,[1037.18799796 1057.32691971 1030.34836416 104...,[-3480.36777739 -3483.25115243 -3479.04902447 ...,[2607.50980467 2593.60701962 2606.01223779 260...,[178.09869982 177.383444 177.47285098 176.77...,[1281.13493694 1270.33904437 1280.55379159 127...
3,9,[-3956.68345173 -3949.48619002 -3941.90894865 ...,[-5704.38870244 -5704.65692337 -5688.51896388 ...,[-2368.21202853 -2362.11000229 -2367.07208956 ...,[-1688.76370058 -1692.11646224 -1686.34971218 ...,[936.18046477 934.99582231 939.13089503 932.44...,[-2864.1525345 -2869.24873224 -2857.20114198 ...,[3038.1161616 3033.75757143 3042.45240002 303...,[-680.29769424 -685.93033384 -676.38613896 -68...,[105.05319894 107.73540827 113.61391707 98.54...,[8726.50101501 8712.12884332 8734.1900151 872...,[5987.09356631 5993.24029603 5995.47547048 598...,[1039.42317241 1064.56888492 1041.43482941 105...,[-3503.74770209 -3513.67187663 -3505.06645501 ...,[2600.69252261 2592.22121146 2604.33585695 259...,[165.60407467 162.81010661 164.82176361 162.81...,[1296.62469585 1287.68399807 1293.94248652 129...
4,6,[-3995.24021092 -3988.86996375 -3987.19358292 ...,[-5737.82691214 -5732.30603126 -5730.74140915 ...,[-2514.10186459 -2513.94540238 -2516.33703903 ...,[-1680.53825862 -1680.18063071 -1677.5654766 ...,[890.71701654 894.38270263 893.53333635 890.22...,[-2843.36541216 -2834.5364731 -2834.31295566 ...,[3018.04429508 3022.40288525 3022.62640269 301...,[-661.07519401 -656.02369976 -654.97316777 -66...,[ -75.05715788 -70.31858806 -75.25832358 -7...,[8799.34535019 8799.59121938 8798.42892866 879...,[5833.76059935 5839.63910814 5832.06186677 583...,[1055.85170458 1063.51835293 1042.26184395 107...,[-3619.35092441 -3623.88832854 -3617.5851366 ...,[2479.05432928 2468.48195416 2478.54023916 246...,[113.97154498 112.9

In [5]:
asl_vis.iloc[0,0]

9

In [6]:
signal_9 = np.array(asl_vis.iloc[0, 1:])
signal_predict = np.array([str_2_list(i) for i in signal_9])

In [7]:
signal_predict.shape

(16, 90)

In [10]:
start = time.time()

# get all feature
current_feature_dict = generate_feature_dict(np.transpose(signal_predict,axes=(1,0)))

# delete no need feature
signal_fe = [current_feature_dict[i] for i in col_name]

clf = joblib.load("./data/trained_ml_models/rf_asl_26class.m")

label = clf.predict([signal_fe])

print("Time for one signal prediction: ", time.time()-start)
print("Pridict label is ", label)

Time for one signal prediction:  1.0158195495605469
Pridict label is  [9]


Label 9 is letter I.

And if you re run the code above, you can find the time changes!

## Conclusion

For more detailed conclusion, please read my internship report, slides or read every notebook carefully.

## Future work

Since my internship was only about five and a half months and computing resources (GPUs) were constrained, we actually had a lot of work that could go on in the future.

For example:
1. In my experiments and code engineering, there are many details that can be improved, and some ideas (I wrote in the notebook) can continue to be tested;
2. Although we are designing personalized AI machines, if more experimenters participate, the upper limit and performance of our model will be greatly improved;
3. Regarding the method of data enhancement, I only use window segmentation for data enhancement. If there is time, it is also a good attempt to use AutoEncoder and GANs for data augmentation and classification.
4. Regarding deep learning, although my task is not good under the deep learning model, there is still potential for improvement, such as training after processing noise, training after enhancing data, adjusting parameters, innovating network structure, etc. You can even try to study the pre-trained EEGtransformer model!

More deeper:

1. We have now completed the visual and imaginative classification of letters, we can go further in brain typing, we can try to combine letters, or we can try to imagine and classify words and sentences directly.
2. We can also design game characters and manipulate them through imagination.


